In [1]:
import torch
import sklearn
import sys
sys.path.append("/mnt/shareb/zliu/analysis/")
sys.path.append("/mnt/shareb/zliu/analysis/CHARMtools")

from CHARMtools import cellcycle
from CHARMtools import CHARMio

import glob
import pandas as pd

from concurrent import futures
from functools import partial

import tqdm

In [2]:
pairs_paths = glob.glob("/mnt/zliu_ssd/CHARM/CHARM_mesc/data/pairs/*")
# sort the paths, consider character+number+character
pairs_paths = sorted(pairs_paths, key=lambda x: int(x.split("/")[-1].split(".")[0].replace("P", "")))
metadata = pd.DataFrame(pairs_paths, columns=["pairs"])
metadata["cellname"] = metadata["pairs"].apply(lambda x: x.split("/")[-1].split(".")[0])

In [3]:
ordering = cellcycle.calc_cellcycle_ordering(metadata,threads=50)
describe = cellcycle.calc_contact_describe(metadata,threads=50)
repliscore = cellcycle.calc_repli_score(metadata,threads=50,repli_chipf = "/mnt/shareb/zliu/analysis/CHARMtools/ref/mm10_repli_chip.wig.gz")
#ordering = ordering.drop(["cellname","pairs"]

/mnt/shareb/zliu/analysis/CHARMtools/mdso/spectral_embedding_.py:173: UserWarning: arpack did not converge. trying lobpcg instead. scale_embedding set to default.
  warnings.warn("arpack did not converge. trying lobpcg instead."
/mnt/shareb/zliu/analysis/CHARMtools/mdso/spectral_embedding_.py:237: UserWarning: Exited at iteration 2000 with accuracies 
[5.55620209e-08 7.07891904e-07 2.15623795e-07 1.79838984e-08
 1.52467303e-08 6.26169575e-09 5.12586604e-09 2.55704721e-10
 6.70833549e-10 3.82513844e-10]
not reaching the requested tolerance 1e-15.
Use iteration 1760 instead with accuracy 
1.3366611798643373e-12.

  lambdas, diffusion_map = lobpcg(laplacian, X, tol=1e-15,
/mnt/shareb/zliu/analysis/CHARMtools/mdso/spectral_embedding_.py:237: UserWarning: Exited postprocessing with accuracies 
[1.64540447e-13 2.32007506e-12 7.36914789e-13 7.60319415e-14
 6.65550845e-14 3.16737621e-13 6.08655513e-13 7.15748519e-14
 1.08991647e-12 2.40795971e-12]
not reaching the requested tolerance 1e-15.
  

In [4]:
metadata = pd.concat([ordering, describe, repliscore], axis=1)

In [5]:
metadata

,pairs,cellname,order_index,near_p,mitotic_p,farAvg,NaganoCellcycle,repli_score,annote_ratio
0,/mnt/zliu_ssd/CHARM/CHARM_mesc/data/pairs/P100...,P1001,361,0.611976,0.149448,2.466807e+07,G1,0.548545,0.992802
1,/mnt/zliu_ssd/CHARM/CHARM_mesc/data/pairs/P100...,P1002,810,0.481156,0.169571,3.655527e+07,G1,0.461432,0.991891
2,/mnt/zliu_ssd/CHARM/CHARM_mesc/data/pairs/P100...,P1003,481,0.600776,0.146153,2.790047e+07,G1,0.534000,0.992275
3,/mnt/zliu_ssd/CHARM/CHARM_mesc/data/pairs/P100...,P1004,387,0.625351,0.155683,2.354133e+07,G1,0.509300,0.992256
4,/mnt/zliu_ssd/CHARM/CHARM_mesc/data/pairs/P100...,P1005,288,0.636014,0.150702,2.476480e+07,early/mid-S,0.527228,0.992273
...,...,...,...,...,...,...,...,...,...
953,/mnt/zliu_ssd/CHARM/CHARM_mesc/data/pairs/P100...,P10092,35,0.689874,0.116459,2.812443e+07,early/mid-S,0.590124,0.993639
954,/mnt/zliu_ssd/CHARM/CHARM_mesc/data/pairs/P100...,P10093,804,0.555538,0.170712,2.936635e+07,G1,0.598628,0.992767
955,/mnt/zliu_ssd/CHARM/CHARM_mesc/data/pairs/P100...,P10094,328,0.744178,0.102277,2.905295e+07,early/mid-S,0.517315,0.994163
956,/mnt/zliu_ssd/CHARM/CHARM_mesc/data/pairs/P100...,P10095,676,0.538673,0.169657,2.374093e+07,G1,0.493892,0.993115


In [6]:
metadata.to_csv("./cc_metadata.tsv", index=False, sep="\t")

In [7]:
with futures.ProcessPoolExecutor(20) as pool:
    cdps = pool.map(cellcycle.dis_counts,pairs_paths)
cdps = pd.DataFrame(list(cdps))
cdps.columns = cdps.columns.astype("string")
cdps.index = metadata["cellname"]

In [8]:
cdps

,0.0,1000.0,1090.5077326652577,1189.2071150027211,1296.8395546510096,1414.213562373095,1542.2108254079408,1681.792830507429,1834.0080864093425,2000.0,...,185363800.04736632,202140657.3078696,220435949.88026974,240387107.90184534,262144000.0,285870059.0718013,311743509.95527333,339958708.21443427,370727600.09473264,404281314.6157392
cellname,,,,,,,,,,,,,,,,,,,,,
P1001,0.0,0.002694,0.002307,0.002304,0.002451,0.002519,0.002590,0.002750,0.002855,0.002949,...,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
P1002,0.0,0.002249,0.001966,0.001950,0.001969,0.001975,0.002185,0.002240,0.002385,0.002558,...,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
P1003,0.0,0.002742,0.002292,0.002439,0.002528,0.002690,0.002811,0.002890,0.003091,0.003170,...,1.215365e-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
P1004,0.0,0.003062,0.002535,0.002507,0.002665,0.002717,0.002851,0.002936,0.003144,0.003305,...,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
P1005,0.0,0.003133,0.002590,0.002550,0.002563,0.002631,0.002825,0.002968,0.003101,0.003252,...,1.013741e-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
P10092,0.0,0.002429,0.002331,0.002336,0.002508,0.002630,0.002663,0.002854,0.002965,0.003149,...,6.450256e-07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
P10093,0.0,0.001686,0.001543,0.001522,0.001639,0.001657,0.001710,0.001873,0.001816,0.001935,...,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
P10094,0.0,0.002260,0.002292,0.002346,0.002548,0.002601,0.002716,0.002880,0.003032,0.003106,...,2.927156e-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
cdps.to_csv("./cdps.tsv", sep="\t")